In [9]:
# Import necessary libraries
import os
import pprint
import torch
import torch.backends.cudnn as cudnn
from models.pidnet import get_seg_model
from configs import config, update_config
from utils.utils import create_logger
from torch.onnx import TrainingMode
# Function to update the configuration
def update_config_for_conversion(cfg_file):
    import argparse
    parser = argparse.ArgumentParser(description='Convert PyTorch model to ONNX')

    parser.add_argument('--cfg', help='experiment configure file name', default=cfg_file, type=str)
    parser.add_argument('--onnx-path', help='path to save the ONNX model', default="model.onnx", type=str)
    parser.add_argument('opts', help="Modify config options using the command-line", default=None, nargs=argparse.REMAINDER)

    args = parser.parse_args(args=[])
    update_config(config, args)

    return args

# Function to load the model
def load_model(cfg, model_file):
    # cudnn related setting
    # cudnn.benchmark = config.CUDNN.BENCHMARK
    # cudnn.deterministic = config.CUDNN.DETERMINISTIC
    # cudnn.enabled = config.CUDNN.ENABLED

    # build model
    model = get_seg_model(cfg, imgnet_pretrained=True)

    pretrained_dict = torch.load(model_file, map_location=torch.device('cpu'))
    if 'state_dict' in pretrained_dict:
        pretrained_dict = pretrained_dict['state_dict']

    model_dict = model.state_dict()
    pretrained_dict = {k[6:]: v for k, v in pretrained_dict.items() if k[6:] in model_dict.keys()}
    model_dict.update(pretrained_dict)
    model.load_state_dict(model_dict)

    return model

# Function to convert the model to ONNX
def convert_to_onnx(model, onnx_path, input_size):
    model = model #.cuda()
    model.eval()

    # prepare dummy input
    dummy_input = torch.randn(3, 3, input_size[1], input_size[0]) #.cuda()

    # Convert model to ONNX format


    # convert to ONNX
    torch.onnx.export(model, dummy_input, onnx_path,
                      export_params=True,
                      opset_version=11,
                      do_constant_folding=True,
                      input_names=['input'],
                      output_names=['output'],
                      training=TrainingMode.TRAINING)

    print(f'Model has been converted to ONNX and saved at {onnx_path}')

# Define paths and configuration file
cfg_file = r"D:\github\PIDNet\configs\cityscapes\pidnet_small_cityscapes_trainval.yaml"
model_file = r"D:\github\PIDNet\pretrained_models\cityscapes\PIDNet_S_Cityscapes_test.pt"  # Change this to your model's path
onnx_path = r"D:\github\PIDNet\pretrained_models\onnx\model.onnx"

# Update the configuration
args = update_config_for_conversion(cfg_file)

# Create logger (if needed)
logger, final_output_dir, _ = create_logger(config, args.cfg, 'test')

# Load the model
model = load_model(config, model_file)

# Convert to ONNX
convert_to_onnx(model, onnx_path, config.TEST.IMAGE_SIZE)


=> creating output\cityscapes\pidnet_small_cityscapes_trainval
=> creating log\cityscapes\pidnet_small\pidnet_small_cityscapes_trainval_2024-07-01-19-04


Attention!!!
Attention!!!
Attention!!!
Attention!!!
Attention!!!
Attention!!!
Attention!!!
Attention!!!
Loaded 302 parameters!
Loaded 302 parameters!
Loaded 302 parameters!
Loaded 302 parameters!
Loaded 302 parameters!
Loaded 302 parameters!
Loaded 302 parameters!
Loaded 302 parameters!
Over!!!
Over!!!
Over!!!
Over!!!
Over!!!
Over!!!
Over!!!
Over!!!


Model has been converted to ONNX and saved at D:\github\PIDNet\pretrained_models\onnx\model.onnx
